# Statistics
Thursday 19 March 2020

Performing statistics on the CNN score results.

In [1]:
%load_ext autoreload
%autoreload 2

In [71]:
from modules import stats
from pandas import read_csv
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, f_oneway
from statsmodels.formula.api import ols
import statsmodels.api as sm

# static variables
CERAD_COLS = ['Cored_MTG', 'Diffuse_MTG', 'CAA_MTG']
TISSUE_SCORE_COLS = ['tissue_cored_score', 'tissue_diffuse_score', 'tissue_caa_score']
GM_SCORE_COLS = ['gm_cored_score', 'gm_diffuse_score', 'gm_caa_score']

# load emory and tang data
emory_data = read_csv('CSVs/Emory_data.csv')
tang_data = read_csv('CSVs/Tang_data.csv')

# subset tang_data to only the hold-out dataset
tang_data = tang_data[tang_data['Group'] == 'hold out'].reset_index(drop=True)

# convert Regan column to int
mapping = {'no': 0, 'low': 1, 'intermediate': 2, 'high': 3}
emory_data = emory_data.replace({'Reagan': mapping})

## Spearman's Rank-order Correlation
Perform Spearman's rank-order correlation between the CNN scores and the ordinal categories. Two ordinal categories are of interest - CERAD-like scores and Reagan Scores.

In [38]:
# * note that this cell deals with CNN scores for the whole tissue
# calculate Spearman's rank order coefficient for the Tang data set
tang_spearman_coef = []
for tissue_col, cerad_col in zip(TISSUE_SCORE_COLS, CERAD_COLS):
    # get column in DataFrame as list
    tissue_scores = tang_data[tissue_col].tolist()
    cerad_scores = tang_data[cerad_col].tolist()
    
    # get coefficient
    rho, p = spearmanr(tissue_scores, cerad_scores)
    tang_spearman_coef.append((rho, p))
    
# calculate the same for the Emory dataset
emory_spearman_coef = []
for tissue_col, cerad_col in zip(TISSUE_SCORE_COLS, CERAD_COLS):
    # get column in DataFrame as list
    tissue_scores = emory_data[tissue_col].tolist()
    cerad_scores = emory_data[cerad_col].tolist()
    
    # get coefficient
    rho, p = spearmanr(tissue_scores, cerad_scores)
    emory_spearman_coef.append((rho, p))

In [45]:
# performing Spearman's rank-order coefficient for Emory data using the gray matter CNN scores
gm_spearman_coef = []
for gm_col, cerad_col in zip(GM_SCORE_COLS, CERAD_COLS):
    # get column in DataFrame as list
    gm_scores = emory_data[gm_col].tolist()
    cerad_scores = emory_data[cerad_col].tolist()
    
    # get coefficient
    rho, p = spearmanr(gm_scores, cerad_scores)
    gm_spearman_coef.append((rho, p))

In [60]:
# repeat with the Reagan scores for emory with tissue and gm scores
reagan_spearman_coef = []
for tissue_col in TISSUE_SCORE_COLS:
    # get column in DataFrame as list
    tissue_scores = emory_data[tissue_col].tolist()
    reagan_scores = emory_data['Reagan'].tolist()
    
    # get coefficient
    rho, p = spearmanr(tissue_scores, reagan_scores)
    reagan_spearman_coef.append((rho, p))
    
reagan_gm_spearman_coef = []
for gm_col in GM_SCORE_COLS:
    # get column in DataFrame as list
    gm_scores = emory_data[gm_col].tolist()
    reagan_scores = emory_data['Reagan'].tolist()
    
    # get coefficient
    rho, p = spearmanr(gm_scores, reagan_scores)
    reagan_gm_spearman_coef.append((rho, p))

## ANOVA
Peform an ANOVA on all comparisons, do following post-hoc tests to identify which groups differ from each other and plot the results. Unlike Spearman's rank-order this analysis can be done on all comparisons in the paper.

References:

https://pythonfordatascience.org/anova-python/

https://stackoverflow.com/questions/16049552/what-statistics-module-for-python-supports-one-way-anova-with-post-hoc-tests-tu

In [65]:
f_oneway(tang_data['tissue_cored_score'][tang_data['Cored_MTG'] == 0],
         tang_data['tissue_cored_score'][tang_data['Cored_MTG'] == 1],
         tang_data['tissue_cored_score'][tang_data['Cored_MTG'] == 2],
         tang_data['tissue_cored_score'][tang_data['Cored_MTG'] == 3]
)

F_onewayResult(statistic=16.453839055526156, pvalue=3.383993724338749e-06)

In [81]:
results = ols('tissue_cored_score ~ C(Cored_MTG)', data=tang_data).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     tissue_cored_score   R-squared:                       0.655
Model:                            OLS   Adj. R-squared:                  0.615
Method:                 Least Squares   F-statistic:                     16.45
Date:                Thu, 19 Mar 2020   Prob (F-statistic):           3.38e-06
Time:                        22:58:18   Log-Likelihood:                 65.736
No. Observations:                  30   AIC:                            -123.5
Df Residuals:                      26   BIC:                            -117.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.0061      0.012      0.517      0.610      -0.018       0.031
C(Cored_MTG)[T.1]     0.0412      0.016      2.626      0.014       0.009       0.073
C(Cored_MTG)[T.2]     0.0865      0.014      6.029      0.000       0.057       0.116
C(Cored_MTG)[T.3]     0.1121      0.021      5.457      0.000       0.070       0.154
==============================================================================
Omnibus:                        0.018   Durbin-Watson:                   1.284
Prob(Omnibus):                  0.991   Jarque-Bera (JB):                0.138
Skew:                           0.049   Prob(JB):                        0.933
Kurtosis:                       2.682   Cond. No.                         5.67
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [72]:
aov_table = sm.stats.anova_lm(results, typ=2)
aov_table

,sum_sq,df,F,PR(>F)
C(Cored_MTG),0.041669,3.0,16.453839,0.000003
Residual,0.021948,26.0,NaN,NaN


In [82]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

mc = MultiComparison(emory_data['tissue_cored_score'], emory_data['Cored_MTG'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     0      1   0.0384 0.0185  0.0051 0.0718   True
     0      2   0.0585  0.001  0.0257 0.0913   True
     0      3    0.113  0.001  0.0608 0.1651   True
     1      2   0.0201 0.3027 -0.0104 0.0506  False
     1      3   0.0745 0.0019  0.0238 0.1253   True
     2      3   0.0544 0.0302   0.004 0.1048   True
---------------------------------------------------


In [83]:
from statsmodels.stats.libqsturng import psturng
import numpy as np

psturng(np.abs(mc_results.meandiffs / mc_results.std_pairs), len(mc_results.groupsunique), mc_results.df_total)

array([0.01851637, 0.001     , 0.001     , 0.30268153, 0.00187439,
       0.0301897 ])